In [1]:
import torch
import torchvision
import gudhi as gd
from gudhi.wasserstein import wasserstein_distance

from scipy.io import loadmat
from sklearn.model_selection import train_test_split
import numpy as np
from copy import deepcopy
from tqdm import tqdm
import math
import matplotlib.pyplot as plt
from scipy.ndimage import distance_transform_edt
%matplotlib inline

from ripser import lower_star_img
from ripser import Rips

import persim
import diagram2vec

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F

import gudhi as gd
from gudhi.wasserstein import wasserstein_distance as dist_w

from IPython.display import clear_output

POT (Python Optimal Transport) package is not installed. Try to run $ conda install -c conda-forge pot ; or $ pip install POT


In [6]:
# mnist dataset from torch
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
mnist_train = torchvision.datasets.MNIST('../data/mnist/mnist_train_data', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.MNIST('../data/mnist/mnist_test_data', train=False, download=True, transform=transform)
X_train = mnist_train.data.numpy()
X_test = mnist_test.data.numpy()
y_train = mnist_train.targets.numpy()
y_test = mnist_test.targets.numpy()

100%|██████████| 9912422/9912422 [00:02<00:00, 4498146.97it/s]


Extracting ../data/mnist/mnist_train_data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/mnist/mnist_train_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 32302851.69it/s]


Extracting ../data/mnist/mnist_train_data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/mnist/mnist_train_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 5558772.64it/s]


...

Extracting ../data/mnist/mnist_test_data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/mnist/mnist_test_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 2361746.82it/s]


Extracting ../data/mnist/mnist_test_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/mnist/mnist_test_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4194304.00it/s]

Extracting ../data/mnist/mnist_test_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/mnist/mnist_test_data/MNIST/raw



In [7]:
class MNIST_Dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx].reshape(28, 28), int(self.y[idx])

In [8]:
batch_size = 8

train_dataset = MNIST_Dataset(X_train, y_train)
test_dataset = MNIST_Dataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [9]:
# save daatsets
torch.save(train_dataset, "../data/mnist/mnist_train.pt")
torch.save(test_dataset, "../data/mnist/mnist_test.pt")

In [10]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
# device = 'cpu'

In [11]:
def diagram(image, device, sublevel=True):
    # get height and square image
    h = int(np.sqrt(image.shape[0]))
    image_sq = image.reshape((h,h))

    # create complex
    cmplx = gd.CubicalComplex(dimensions=(h, h), top_dimensional_cells=(sublevel*image))

    # get pairs of critical simplices
    cmplx.compute_persistence()
    critical_pairs = cmplx.cofaces_of_persistence_pairs()
    
    # get essential critical pixel
    bpx0_essential = critical_pairs[1][0][0] // h, critical_pairs[1][0][0] % h

    # get critical pixels corresponding to critical simplices
    try:
        bpx0 = [critical_pairs[0][0][i][0] for i in range(len(critical_pairs[0][0]))]
        dpx0 = [critical_pairs[0][0][i][1] for i in range(len(critical_pairs[0][0]))]
    except IndexError:
        bpx0 = []
        dpx0 = []
        
    try:
        bpx1 = [critical_pairs[0][1][i][0] for i in range(len(critical_pairs[0][1]))]
        dpx1 = [critical_pairs[0][1][i][1] for i in range(len(critical_pairs[0][1]))]
    except IndexError:
        bpx1 = []
        dpx1 = []
    

    flat_image = image_sq.flatten()
    pd0_essential = torch.tensor([[image_sq[bpx0_essential], torch.max(image)]])

    if (len(bpx0)!=0):
        pdb0 = flat_image[bpx0][:, None]
        pdd0 = flat_image[dpx0][:, None]
        pd0 = torch.Tensor(torch.hstack([pdb0, pdd0]))
        pd0 = torch.vstack([pd0, pd0_essential.to(device)])
    else:
        pd0 = pd0_essential

    if (len(bpx1)!=0):
        pdb1 = flat_image[bpx1][:, None]
        pdd1 = flat_image[dpx1][:, None]
        pd1 = torch.Tensor(torch.hstack([pdb1, pdd1]))
    else:
        pd1 = torch.zeros((1, 2))
    
    return pd0, pd1

class ConvDiagram(nn.Module):
    def __init__(self, device):
        super(ConvDiagram, self).__init__()
        self.device = device
        
    def forward(self, x):
        diagrams = []
        for i in range(x.shape[0]):
            res = diagram(x[i].flatten(), self.device)
            for j in range(len(res)):
                diagrams.append(torch.concatenate([res[j], torch.Tensor([[j, i] for _ in range(res[j].shape[0])]).to(self.device)], axis=1))
        diagrams = torch.concatenate(diagrams)
        return diagrams

In [12]:
class Transformer(torch.nn.Module):
    def __init__(self, n_in, n_hidden, n_out, seq_size=1024, nhead=2, num_layers=2, dim_feedforward=16):
        super(Transformer, self).__init__()
        self.embeddings = nn.Linear(n_in, n_hidden)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=n_hidden, nhead=nhead, dim_feedforward=dim_feedforward, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer=self.encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(seq_size, n_out)

    def forward(self, X):
        X = self.embeddings(X)
        X = self.transformer(X)
        X = X.mean(dim=-1)
        X = self.classifier(X)
        X = X.softmax(dim=-1)
        return X

In [13]:
class TopologicalConvTransformer(nn.Module):
    def __init__(self, n_in, n_conv, max_sequence, n_diag, n_hidden, n_out, nhead=2, num_layers=2, dim_feedforward=16, device='cuda'):
        super(TopologicalConvTransformer, self).__init__()
        
        self.max_sequence = max_sequence
        self.conv = nn.Conv2d(n_in, n_conv, 3)
        self.diagram = ConvDiagram(device)
        self.transformer = Transformer(n_diag, n_hidden, n_out, max_sequence, nhead, num_layers, dim_feedforward)

    def forward(self, xs):
        result = []
        for i in range(xs.shape[0]):
            x = xs[i][None, :, :] / 256
            x = self.conv(x)
            x = self.diagram(x)
            if x.shape[0] > self.max_sequence:
                x = x[:self.max_sequence]
            x = F.pad(x, (0, 0, 0, self.max_sequence - x.shape[0]), "constant", 0)
            x = self.transformer(x)
            result.append(x[None, :])
        result = torch.concatenate(result, axis=0)
        return result
        

In [14]:
class BaselineDiagram(nn.Module):
    def __init__(self, device):
        super(BaselineDiagram, self).__init__()
        self.device = device
    
    def forward(self, x):
        diagrams = []
        res = diagram(torch.Tensor(x.flatten()), self.device)
        for j in range(len(res)):
            if not res[j].shape[0]:
                diagrams.append(torch.zeros(0, 4))
            else:
                diagrams.append(torch.concatenate([res[j], torch.Tensor([[j, 1] for _ in range(res[j].shape[0])])], axis=1))
        diagrams = torch.concatenate(diagrams)
        return diagrams

class TopologicalBaselineTransformer(nn.Module):
    def __init__(self, max_sequence, n_diag, n_hidden, n_out, nhead=2, num_layers=2, dim_feedforward=16, device='cuda'):
        super(TopologicalBaselineTransformer, self).__init__()
        
        self.max_sequence = max_sequence
        self.diagram = BaselineDiagram(device)
        self.transformer = Transformer(n_diag, n_hidden, n_out, max_sequence, nhead, num_layers, dim_feedforward)
    
    def forward(self, xs):
        result = []
        for i in range(xs.shape[0]):
            x = xs[i][None, :, :] / 256
            x = self.diagram(x)
            if x.shape[0] > self.max_sequence:
                x = x[:self.max_sequence]
            x = F.pad(x, (0, 0, 0, self.max_sequence - x.shape[0]), "constant", 0)
            x = self.transformer(x)
            result.append(x[None, :])
        result = torch.concatenate(result, axis=0)
        return result

In [15]:
class CedtDiagram(nn.Module):
    def __init__(self, device):
        super(CedtDiagram, self).__init__()
        self.device = device
    
    def forward_baseline(self, x):
        diagrams = []
        res = diagram(torch.Tensor(x.flatten()), self.device)
        for j in range(len(res)):
            if not res[j].shape[0]:
                diagrams.append(torch.zeros(0, 4))
            else:
                diagrams.append(torch.concatenate([res[j], torch.Tensor([[j, 1] for _ in range(res[j].shape[0])])], axis=1))
        diagrams = torch.concatenate(diagrams)
        return diagrams

    def forward(self, x):
        edt = torch.Tensor(distance_transform_edt(x > 0.5).reshape(1, x.shape[1], x.shape[2]))
        cedt = edt * (x > 0.5) - edt * (x <= 0.5)
        return self.forward_baseline(cedt)

class TopologicalCedtTransformer(nn.Module):
    def __init__(self, max_sequence, n_diag, n_hidden, n_out, nhead=2, num_layers=2, dim_feedforward=16, device='cuda'):
        super(TopologicalCedtTransformer, self).__init__()
        
        self.max_sequence = max_sequence
        self.diagram = CedtDiagram(device)
        self.transformer = Transformer(n_diag, n_hidden, n_out, max_sequence, nhead, num_layers, dim_feedforward)
    
    def forward(self, xs):
        result = []
        for i in range(xs.shape[0]):
            x = xs[i][None, :, :] / 256
            x = self.diagram(x)
            if x.shape[0] > self.max_sequence:
                x = x[:self.max_sequence]
            x = F.pad(x, (0, 0, 0, self.max_sequence - x.shape[0]), "constant", 0)
            x = self.transformer(x)
            result.append(x[None, :])
        result = torch.concatenate(result, axis=0)
        return result

In [16]:
class DirectionalDiagram(nn.Module):
    def __init__(self, filter_count, device):
        super(DirectionalDiagram, self).__init__()
        self.device = device
        self.filters = nn.Parameter(torch.randn(filter_count) * 2 * math.pi)

    def process_by_direction(self, img, alpha):
        X = (math.cos(alpha) - (np.arange(0, img.shape[0]) - (img.shape[0] / 2 - 0.5)) / (img.shape[0] * math.sqrt(2))) * math.cos(alpha) / 2
        Y = (math.sin(alpha) - (np.arange(0, img.shape[1]) - (img.shape[1] / 2 - 0.5)) / (img.shape[1] * math.sqrt(2))) * math.sin(alpha) / 2
        direction_filter = X.reshape(-1, 1) + Y.reshape(1, -1)
        return torch.Tensor(direction_filter) + img

    def forward(self, x):
        w = int(np.sqrt(x.flatten().shape[0]))
        imgs = [self.process_by_direction(x.reshape(w, w), alpha) for alpha in self.filters]
        diagrams = []
        for i, img in enumerate(imgs):
            res = diagram(torch.Tensor(img.flatten()), self.device)
            for j in range(len(res)):
                if not res[j].shape[0]:
                    diagrams.append(torch.zeros(0, 4))
                else:
                    diagrams.append(torch.concatenate([res[j], torch.Tensor([[j, self.filters[i]] for _ in range(res[j].shape[0])])], axis=1))

        diagrams = torch.concatenate(diagrams)
        return diagrams

class TopologicalDirectionalTransformer(nn.Module):
    def __init__(self, filter_count, max_sequence, n_diag, n_hidden, n_out, nhead=2, num_layers=2, dim_feedforward=16, device='cuda'):
        super(TopologicalDirectionalTransformer, self).__init__()
        
        self.max_sequence = max_sequence
        self.diagram = DirectionalDiagram(filter_count, device)
        self.transformer = Transformer(n_diag, n_hidden, n_out, max_sequence, nhead, num_layers, dim_feedforward)
    
    def forward(self, xs):
        result = []
        for i in range(xs.shape[0]):
            x = xs[i][None, :, :] / 256
            x = self.diagram(x)
            if x.shape[0] > self.max_sequence:
                x = x[:self.max_sequence]
            x = F.pad(x, (0, 0, 0, self.max_sequence - x.shape[0]), "constant", 0)
            x = self.transformer(x)
            result.append(x[None, :])
        result = torch.concatenate(result, axis=0)
        return result

In [17]:
CLASS_NUM = 10

def fit(train_dataloader, test_dataloader, model, criterion, optimizer, epochs):
    best_model = deepcopy(model)
    best_loss = np.inf
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []
    for epoch in range(epochs):
        model.train()
        cur_train_loss = []
        cur_test_loss = []
        cur_train_accuracy = []
        cur_test_accuracy = []
        for batch in tqdm(train_dataloader):
            optimizer.zero_grad()
            target = batch[1].long().to(device)
            output = model(batch[0].to(device))
            loss = criterion(output, F.one_hot(target, CLASS_NUM).float())
            loss.backward()
            optimizer.step()
            cur_train_loss.append(loss.item())
            cur_train_accuracy.append((output.argmax(-1) == target).float().mean().item())

        with torch.no_grad():
            for batch in tqdm(test_dataloader):
                target = batch[1].long().to(device)
                output = model(batch[0].to(device))
                loss = criterion(output, F.one_hot(target, CLASS_NUM).float())
                cur_test_loss.append(loss.item())
                cur_test_accuracy.append((output.argmax(-1) == target).float().mean().item())

        clear_output()

        train_losses.append(np.mean(np.array(cur_train_loss)))
        test_losses.append(np.mean(np.array(cur_test_loss)))
        train_accuracies.append(np.mean(np.array(cur_train_accuracy)))
        test_accuracies.append(np.mean(np.array(cur_test_accuracy)))

        print(f'{epoch}: train_accuracy={train_accuracies[-1]}, test_accuracy={test_accuracies[-1]}, train_loss={train_losses[-1]}, test_loss={test_losses[-1]}')
        if test_losses[-1] < best_loss:
            print(f'Save the new best model')
            best_model = deepcopy(model)
            best_loss = test_losses[-1]

        plt.subplot(1, 2, 1)
        plt.plot(np.arange(epoch+1), test_accuracies, label='test_accuracy')
        plt.plot(np.arange(epoch+1), train_accuracies, label='train_accuracy')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(np.arange(epoch+1), test_losses, label='test_loss')
        plt.plot(np.arange(epoch+1), train_losses, label='train_loss')
        plt.legend()

        plt.show()
    return best_model

In [18]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device = 'cpu'
device = None

In [19]:
model = TopologicalDirectionalTransformer(filter_count=4, max_sequence=256, n_diag=4, n_hidden=32, n_out=10, nhead=4, num_layers=2, dim_feedforward=16, device='cpu').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999))
criterion = nn.BCELoss()
best_model = fit(train_loader, test_loader, model, criterion, optimizer, 100)

  0%|          | 0/7500 [00:00<?, ?it/s]

  0%|          | 37/7500 [00:02<10:01, 12.41it/s]


KeyboardInterrupt: 

In [47]:
model = TopologicalCedtTransformer(max_sequence=256, n_diag=4, n_hidden=32, n_out=10, nhead=4, num_layers=2, dim_feedforward=16, device='cpu').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999))
criterion = nn.BCELoss()
best_model = fit(train_loader, test_loader, model, criterion, optimizer, 100)

  1%|          | 81/7000 [00:04<06:10, 18.68it/s]


KeyboardInterrupt: 

In [24]:
model = TopologicalBaselineTransformer(max_sequence=256, n_diag=4, n_hidden=32, n_out=10, nhead=4, num_layers=2, dim_feedforward=16, device='cpu').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999))
criterion = nn.BCELoss()
best_model = fit(train_loader, test_loader, model, criterion, optimizer, 100)

  2%|▏         | 118/7000 [00:05<05:49, 19.71it/s]


KeyboardInterrupt: 

In [121]:
model = TopologicalConvTransformer(n_in=1, n_conv=4, max_sequence=256, n_diag=4, n_hidden=32, n_out=10, nhead=4, num_layers=2, dim_feedforward=16, device='cpu').to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, betas=(0.9, 0.999))
criterion = nn.BCELoss()
best_model = fit(train_loader, test_loader, model, criterion, optimizer, 100)

  0%|          | 29/7000 [00:02<09:49, 11.83it/s]


KeyboardInterrupt: 